Double elastic pendulum! Elastic-ness serves as a soft-constraint, so no need for hard-constraint integration

Imports

In [ ]:
from dataclasses import dataclass
from typing import Tuple

import numpy as np
import matplotlib.pyplot as plt

Define simulation class that performs integration

In [ ]:
def normalize(x: np.typing.NDArray) -> np.typing.NDArray:

    return x / np.linalg.norm(x)


@dataclass
class DoublePendulum:

    acceleration_from_gravity: float
    spring_constant: float
    mass: float
    length: float
    anchor: np.typing.NDArray
    positions: Tuple[np.typing.NDArray, np.typing.NDArray]
    velocities: Tuple[np.typing.NDArray, np.typing.NDArray]

    def __post_init__(self):

        if not isinstance(self.anchor, np.ndarray):
            self.anchor = np.array(self.anchor)

        if not isinstance(self.positions, np.ndarray):
            self.positions = np.array(self.positions)

        if not isinstance(self.velocities, np.ndarray):
            self.velocities = np.array(self.velocities)

    @property
    def gravity(self) -> float:

        return np.array([0, self.mass * self.acceleration_from_gravity])

    @property
    def potential_energy(self) -> float:

        return self.acceleration_from_gravity * self.mass * (self.positions[0][1] + self.positions[1][1]) + \
               0.5 * self.spring_constant * (
                       (np.linalg.norm(self.positions[0] - self.anchor) - self.length) ** 2 +
                       (np.linalg.norm(self.positions[1] - self.positions[0]) - self.length) ** 2
               )

    @property
    def kinetic_energy(self) -> float:

        return 0.5 * self.mass * (np.linalg.norm(self.velocities[0]) ** 2 + np.linalg.norm(self.velocities[1]) ** 2)

    def step(self, timestep: float):

        r1, r2 = self.positions
        v1, v2 = self.velocities

        f1 = self.spring_constant * (np.linalg.norm(self.anchor - r1) - self.length) * normalize(self.anchor - r1) + \
            self.spring_constant * (np.linalg.norm(r2 - r1) - self.length) * normalize(r2 - r1) - self.gravity
        f2 = self.spring_constant * (np.linalg.norm(r1 - r2) - self.length) * normalize(r1 - r2) - self.gravity

        # Velocity Verlet: half-step velocity
        v1_half = v1 + 0.5 * timestep * f1
        v2_half = v2 + 0.5 * timestep * f2

        # Position update
        r1 += timestep * v1_half
        r2 += timestep * v2_half

        # New forces
        f1_new = self.spring_constant * (np.linalg.norm(self.anchor - r1) - self.length) * normalize(self.anchor - r1) + \
                self.spring_constant * (np.linalg.norm(r2 - r1) - self.length) * normalize(r2 - r1) - self.gravity
        f2_new = self.spring_constant * (np.linalg.norm(r1 - r2) - self.length) * normalize(r1 - r2) - self.gravity

        # Final velocity update
        v1 = v1_half + 0.5 * timestep * f1_new / self.mass
        v2 = v2_half + 0.5 * timestep * f2_new / self.mass

        self.positions = (r1, r2)
        self.velocities = (v1, v2)

Example run

In [ ]:
double_pendulum = DoublePendulum(
    acceleration_from_gravity=9.81,
    spring_constant=50.0,
    length=1.0,
    mass=1.0,
    anchor=(0.0, 0.0),
    positions=[
        [1.0, 0.0],
        [2.0, 0.0],
    ],
    velocities=[
        [0.0, 0.0],
        [0.0, 0.0]
    ]
)


r1_history = []
r2_history = []
KE_history = []
PE_history = []

steps = 10000
timestep = 1.0e-3
for _ in range(steps):
    r1_history.append(double_pendulum.positions[0])
    r2_history.append(double_pendulum.positions[1])
    KE_history.append(double_pendulum.kinetic_energy)
    PE_history.append(double_pendulum.potential_energy)

    double_pendulum.step(timestep=timestep)

# Plot energy over time
KE_history = np.array(KE_history)
PE_history = np.array(PE_history)
t = np.linspace(0, timestep * steps, steps)
plt.plot(t, KE_history, label="kinetic")
plt.plot(t, PE_history, label="potential")
plt.plot(t, KE_history + PE_history, label="total")
plt.legend()
plt.xlabel('Time')
plt.ylabel('Energy')
plt.title('Energy Conservation in Double Elastic Pendulum')
plt.grid()
plt.show()